In [1]:
import keras
from keras import layers
from keras import models
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [3]:
imageSet=[]
controllerSet=[]
leftSet=[]
rightSet=[]
leftBoostSet=[]
rightBoostSet=[]
boostSet=[]
noneActionSet=[]
IMAGE_DIMS = [15, 15, 1]
cwd = Path().resolve()

In [4]:
# import the necessary packages
import keras
from keras import layers
from keras import models


class SmallerVGGNet:
	@staticmethod
	def build(width, height, depth):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = models.Sequential()
		# CONV => RELU => POOL
		model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, depth)))
		model.add(layers.MaxPooling2D((2, 2)))
		model.add(layers.Conv2D(64, (3, 3), activation='relu'))
		model.add(layers.MaxPooling2D((2, 2)))
		# model.summary()
		# softmax classifier
		model.add(layers.Flatten())
		model.add(layers.Dense(64, activation='relu'))
		model.add(layers.Dense(5, activation='softmax'))  # softmax layer
		# return the constructed network architecture
		return model


In [5]:
# add more epoch
def KerasPreprocessing():
	data = np.array(imageSet, dtype="float") / 255.0
	data = data.reshape(len(data), 15, 15)
	labels = classifyController()
	labels = np.array(labels)
	print("Data.shape ",data.shape, "________ Label shape: ", labels.shape)
	lb = LabelBinarizer()
	labels = lb.fit_transform(labels)
	(train_images, test_images, train_labels, test_labels) = train_test_split(data, labels, test_size=0.2, random_state=42)
	print("train_images: ",len(train_images), "test_images :",len(test_images),"train_labels :", len(train_labels), "test_labels :",len(test_labels))
	train_images = np.expand_dims(train_images, axis=3)
	test_images = np.expand_dims(test_images, axis=3)
	model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0], depth=IMAGE_DIMS[2])
	model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
	history = model.fit(train_images, train_labels, validation_split=0.2, epochs=20, batch_size=20)
	test_loss, test_acc = model.evaluate(test_images, test_labels)
	print("test_acc :", test_acc)
	showModel(history)
    
def isNullAction(list):
	for i in list:
		if i != 0:
			return False
	return True

def classify(image,controller):  # Deprecated only for validate Data
	global leftSet,rightSet,leftBoostSet,rightBoostSet,boostSet
	if controller[1]==1:        #boost
		if controller[6]==1:
			leftBoostSet.append(image)
		elif controller[7]==1:
			rightBoostSet.append(image)
		else:
			boostSet.append(image)
	elif controller[6]==1:
		leftSet.append(image)
	elif controller[7]==1:
		rightSet.append(image)
	else:
		noneActionSet.append(image)

def classifyController():
	labels=[]
	for controller in controllerSet:
		if controller[1]==1:
			if controller[6]==1:
				labels.append("LB")
			elif controller[7]==1:
				labels.append("RB")
			else:
				labels.append("B")
		elif controller[6]==1:
			labels.append("L")
		elif controller[7]==1:
			labels.append("R")
		else:
			labels.append("")
	return labels        

def optimiseData():
	global imageSet,controllerSet
	fast = 0
	slow = 0
	for fast in range(len(imageSet)):
		if(not isNullAction(controllerSet[fast])):
			imageSet[slow] = imageSet[fast]
			controllerSet[slow] = controllerSet[fast]
			slow+=1;
			fast+=1;
		else:
			fast+=1;
	imageSet = imageSet[0 : slow]
	controllerSet = controllerSet[0 : slow]
	for (x,y) in zip(imageSet,controllerSet):
		classify(x,y)
	# print(optimisedLabel)
	print("noneActionSet",len(noneActionSet))
	print("leftSet",len(leftSet))
	print("leftBoostSet",len(leftBoostSet))
	print("rightSet",len(rightSet))
	print("rightBoostSet",len(rightBoostSet))
	print("boostSet",len(boostSet))

def ParseFullData(FileName):  #tradition way
	global imageSet,controllerSet
	module_path = cwd 
	dataFileName = str(module_path) + FileName
	with open(dataFileName, "r") as f:
		x=0
		for line in f.readlines():
			line=line.strip()
			if x%2==0:
				imageSet.append(list(map(int,line.split(" "))))
			else:
				controllerSet.append(list(map(int,line.split(" "))))
			x+=1
	# for i in range(len(imageSet)):     	# find the Wrong Data
	# 	if len(imageSet[i]) != 225:
	# 		print(i,"   ",len(imageSet[i]))
	# print(len(imageSet))
	optimiseData()
	KerasPreprocessing()

In [6]:
def initModel():
	model = models.Sequential()
	# CONV => RELU => POOL
	model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(15, 15, 1)))
	model.add(layers.MaxPooling2D((2, 2)))
	model.add(layers.Conv2D(64, (3, 3), activation='relu'))
	model.add(layers.MaxPooling2D((2, 2)))
	# model.summary()
	# softmax classifier
	model.add(layers.Flatten())
	model.add(layers.Dense(64, activation='relu'))
	model.add(layers.Dense(5, activation='softmax'))
	return model

def showModel(history):
	plt.plot(history.history['acc'])
	plt.plot(history.history['val_acc'])
	plt.title('Model accuracy')
	plt.ylabel('Accuracy')
	plt.xlabel('Epoch')
	plt.legend(['Train', 'Test'], loc='upper left')
# 	plt.savefig(module_path + "/MatplotImages/ModelAccuracy" + helper.getNameByTime() + ".png")
	plt.show()
	# Plot training & validation loss values
	plt.plot(history.history['loss'])
	plt.plot(history.history['val_loss'])
	plt.title('Model loss')
	plt.ylabel('Loss')
	plt.xlabel('Epoch')
	plt.legend(['Train', 'Test'], loc='upper left')
# 	plt.savefig(module_path + "/MatplotImages/ModelLoss" + helper.getNameByTime() + ".png")
	plt.show()  

## Add more epochs and add a pooling layer

In [ ]:
# comment it all to accumulate(copy) the data
imageSet=[]
controllerSet=[]
leftSet=[]
rightSet=[]
leftBoostSet=[]
rightBoostSet=[]
boostSet=[]
noneActionSet=[]
ParseFullData("/FullData2.txt")

noneActionSet 0
leftSet 240
leftBoostSet 281
rightSet 105
rightBoostSet 261
boostSet 1056
Data.shape  (1943, 15, 15) ________ Label shape:  (1943,)
train_images:  1554 test_images : 389 train_labels : 1554 test_labels : 389
Train on 1243 samples, validate on 311 samples
Epoch 1/20
1243/1243 [==============================] - 11s 9ms/step - loss: 1.3030 - acc: 0.5117 - val_loss: 1.1354 - val_acc: 0.5884
Epoch 2/20
1243/1243 [==============================] - 2s 2ms/step - loss: 1.1006 - acc: 0.5744 - val_loss: 0.9972 - val_acc: 0.6206
Epoch 3/20
1243/1243 [==============================] - 2s 1ms/step - loss: 1.0117 - acc: 0.5929 - val_loss: 0.9728 - val_acc: 0.6141
Epoch 4/20
 960/1243 [======================>.......] - ETA: 0s - loss: 0.9737 - acc: 0.6042